![](http://www.dataivy.cn/book/images/head.png)
<table align="center" bgcolor="#FFFFFF" border="0px">
   <tr bgcolor="#FFFFFF">
      <td><img src="http://www.dataivy.cn/book/images/release_date.svg"></td>
      <td><img src="http://www.dataivy.cn/book/images/python-3.7-green.svg"></td>
      <td><a href="http://www.dataivy.cn/blog/python_book_faq/"><img src="http://www.dataivy.cn/book/images/faq-visit_site-blue.svg"></a></td>
      <td><a href="http://www.dataivy.cn/blog/python_book_knows_issues/"><img src="http://www.dataivy.cn/book/images/known_issues.svg"></a></td>
   </tr>
</table>


<hr />

<div align="center"><h1> 第八章&nbsp;&nbsp;内容数据化运营</h1></div>

# 一、案例-基于潜在狄利克雷分配（LDA）的内容主题挖掘

## 说明

- 时间：2019-01-01
- 作者：宋天龙（Tony Song）
- 程序开发环境：win7 64位
- Python版本：64位 3.7
- 依赖库：tarfile、os、jieba、gensim、bs4
- 程序输入：article.txt、news_data.tar.gz
- 程序输出：打印输出18个主题及新文本的预测主题归属

## 程序

### 导入库

In [1]:
import os
import tarfile

import jieba.posseg as pseg
from bs4 import BeautifulSoup
from gensim import corpora, models

C:\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### 函数模块

In [2]:
# 中文分词
def jieba_cut(text):
    '''
    将输入的文本句子根据词性标注做分词
    :param text: 文本句子，字符串型
    :return: 符合规则的分词结果
    '''
    rule_words = ['z', 'vn', 'v', 't', 'nz', 'nr', 'ns', 'n', 'l', 'i', 'j', 'an','a']
    words = pseg.cut(text)
    seg_list = [word.word for word in words if word.flag in rule_words]
    return seg_list

In [3]:
# 文本预处理
def text_pro(words_list, tfidf_object=None, training=True):
    '''
    gensim主题建模预处理过程，包含分词类别转字典、生成语料库和TF-IDF转换
    :param words_list: 分词列表，列表型
    :param tfidf_object: TF-IDF模型对象，该对象在训练阶段生成
    :param training: 是否训练阶段，用来针对训练和预测两个阶段做预处理
    :return: 如果是训练阶段，返回词典、TF-IDF对象和TF-IDF向量空间数据；如果是预测阶段，返回TF-IDF向量空间数据
    '''
    # 分词列表转字典
    dic = corpora.Dictionary(words_list)  # 将分词列表转换为字典形式
    print('{:*^60}'.format('token & word mapping review:'))
    for i, w in list(dic.items())[:5]:  # 循环读出字典前5条的每个key和value，对应的是索引值和分词
        print('token:%s -- word:%s' % (i, w))
    # 生成语料库
    corpus = [dic.doc2bow(words) for words in words_list]  # 用于存储语料库的列表
    print('{:*^60}'.format('bag of words review:'))
    print(corpus[0])
    # TF-IDF转换
    if training:
        tfidf = models.TfidfModel(corpus)  # 建立TF-IDF模型对象
        corpus_tfidf = tfidf[corpus]  # 得到TF-IDF向量稀疏矩阵
        print('{:*^60}'.format('TF-IDF model review:'))
        print(list(corpus_tfidf)[0])  # 打印第一条向量
        return dic, corpus_tfidf, tfidf
    else:
        return tfidf_object[corpus]

In [4]:
# 全角转半角
def str_convert(content):
    '''
    将内容中的全角字符，包含英文字母、数字键、符号等转换为半角字符
    :param content: 要转换的字符串内容
    :return: 转换后的半角字符串
    '''
    strs = []
    for each_char in content:  # 循环读取每个字符
        code_num = ord(each_char)  # 读取字符的ASCII值或Unicode值
        if code_num == 12288:  # 全角空格直接转换
            code_num = 32
        elif 65281 <= code_num <= 65374:  # 全角字符（除空格）根据关系转化
            code_num -= 65248
        strs.append(chr(code_num))
    return ''.join(strs)

In [5]:
# 解析文件内容
def data_parse(data):
    '''
    从原始文件中解析出文本内容数据
    :param data: 包含代码的原始内容
    :return: 文本中的所有内容，列表型
    '''
    raw_code = BeautifulSoup(data, 'lxml')  # 建立BeautifulSoup对象
    content_code = raw_code.find_all('content')  # 从包含文本的代码块中找到content标签
    content_list = [str_convert(each_content.text) for each_content in content_code if len(each_content) > 0]
    return content_list

### 读取数据并解析

In [6]:
# 解压缩文件
if not os.path.exists('./news_data'):  # 如果不存在数据目录，则先解压数据文件
    with tarfile.open('news_data.tar.gz') as tar:  # 打开tar.gz压缩包对象
        names = tar.getnames()  # 获得压缩包内的每个文件对象的名称
        for name in names:  # 循环读出每个文件
            tar.extract(name, path='./')  # 将文件解压到指定目录

In [7]:
# 汇总所有内容
all_content = []  # 总列表，用于存储所有文件的文本内容
for root, dirs, files in os.walk('./news_data'):  # 分别读取遍历目录下的根目录、子目录和文件列表
    for file in files:  # 读取每个文件
        file_name = os.path.join(root, file)  # 将目录路径与文件名合并为带有完整路径的文件名
        with open(file_name, encoding='utf-8') as f:  # 以只读方式打开文件
            data = f.read()  # 读取文件内容
        all_content.extend(data_parse(data))  # 从文件内容中获取文本并将结果追加到总列表

### 分词

In [8]:
print('get word list...')
words_list = [list(jieba_cut(each_content)) for each_content in all_content]  # 分词列表，用于存储所有文件的分词结果

Building prefix dict from the default dictionary ...
Loading model from cache D:\system_backup\Temp\jieba.cache


get word list...


Loading model cost 0.847 seconds.
Prefix dict has been built succesfully.


### 建立主题模型

In [22]:
dic, corpus_tfidf, tfidf = text_pro(words_list)  # 训练集的文本预处理
num_topics = 3  # 设置主题个数
lda = models.LdaModel(corpus_tfidf, id2word=dic, num_topics=num_topics)  # 通过LDA进行主题建模
print('{:*^60}'.format('topic model review:'))
print(lda.print_topics())

****************token & word mapping review:****************
token:0 -- word:仇恨
token:1 -- word:侮辱
token:2 -- word:侵害
token:3 -- word:凶杀
token:4 -- word:危害
********************bag of words review:********************
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)]
********************TF-IDF model review:********************
[(0, 0.16762633852828174), (1, 0.16660204914253687), (2, 0.1643986382302142), (3, 0.168282481745965), (4, 0.16197667368712637), (5, 0.14602961468426073), (6, 0.16282320045073903), (7, 0.10154448591145282), (8, 0.12365275311464316), (9, 0.12399080729729553), (10, 0.16703117734810868), (11, 0.163124879458702), (12, 0.16844765669812112), (13, 0.16409043499326897), (14, 0.1662290891913951), (15, 0.1685028172752526), (16, 0.332245916102828), (1

### 新数据集的主题模型预测

In [23]:
with open('article.txt', encoding='utf-8') as f:  # 打开新的文本
    text_new = f.read()  # 读取文本数据
text_content = data_parse(data)  # 解析新的文本
words_list_new = jieba_cut(text_new)  # 将文本转换为分词列表
corpus_tfidf_new = text_pro([words_list_new], tfidf_object=tfidf, training=False)  # 新文本数据集的预处理
corpus_lda_new = lda[corpus_tfidf_new]  # 获取新的分词列表（文档）的主题概率分布
print('{:*^60}'.format('topic forecast:'))
print(list(corpus_lda_new))

****************token & word mapping review:****************
token:0 -- word:一鸣惊人
token:1 -- word:三剑客
token:2 -- word:上演
token:3 -- word:不败
token:4 -- word:专业培训
********************bag of words review:********************
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 3), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 2), (17, 1), (18, 1), (19, 3), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 3), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 2), (36, 1), (37, 2), (38, 1), (39, 1), (40, 2), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 2), (55, 3), (56, 1), (57, 1), (58, 1), (59, 2), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 2), (71, 1), (72, 1), (73, 4), (74, 1), (75, 1), (76, 1), (77, 7), (78, 5), (79, 2), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87,

# 二、案例-基于多项式贝叶斯的增量学习的文本分类

## 说明

- 时间：2019-01-01
- 作者：宋天龙（Tony Song）
- 程序开发环境：win7 64位
- Python版本：64位 3.7
- 依赖库：re、tarfile、os、numpy、bs4、sklearn
- 程序输入：article.txt、news_data.tar.gz
- 程序输出：打印输出新的内容所属的主题信息

## 程序

### 导入库

In [11]:
import os
import tarfile
import re
import numpy as np
from bs4 import BeautifulSoup  # 用于XML格式化处理
from sklearn.feature_extraction.text import HashingVectorizer  # 文本转稀疏矩阵
from sklearn.naive_bayes import MultinomialNB  # 贝叶斯分类器
from sklearn.metrics import accuracy_score  # 分类评估指标

### 函数模块

In [12]:
# 全角转半角
def str_convert(content):
    '''
    将内容中的全角字符，包含英文字母、数字键、符号等转换为半角字符
    :param content: 要转换的字符串内容
    :return: 转换后的半角字符串
    '''
    strs = []
    for each_char in content:  # 循环读取每个字符
        code_num = ord(each_char)  # 读取字符的ASCII值或Unicode值
        if code_num == 12288:  # 全角空格直接转换
            code_num = 32
        elif 65281 <= code_num <= 65374:  # 全角字符（除空格）根据关系转化
            code_num -= 65248
        strs.append(chr(code_num))
    return ''.join(strs)

In [13]:
# 解析文件内容
def data_parse(data):
    '''
    从原始文件中解析出文本内容和标签数据
    :param data: 包含代码的原始内容
    :return: 以列表形式返回文本中的所有内容和对应标签
    '''
    raw_code = BeautifulSoup(data, "lxml")  # 建立BeautifulSoup对象
    doc_code = raw_code.find_all('doc')  # 从包含文本的代码块中找到doc标签
    content_list = []  # 建立空列表，用来存储每个content标签的内容
    label_list = []  # 建立空列表，用来存储每个content对应的label的内容
    for each_doc in doc_code:  # 循环读出每个doc标签
        if len(each_doc) > 0:  # 如果dco标签的内容不为空
            content_code = each_doc.find('content')  # 从包含文本的代码块中找到doc标签
            raw_content = content_code.text  # 获取原始内容字符串
            convert_content = str_convert(raw_content)  # 将全角转换为半角
            content_list.append(convert_content)  # 将content文本内容加入列表

            label_code = each_doc.find('url')  # 从包含文本的代码块中找到url标签
            label_content = label_code.text  # 获取url信息
            label = re.split('[/|.]', label_content)[2]  # 将URL做分割并提取子域名
            label_list.append(label)  # 将子域名加入列表
    return content_list, label_list

In [14]:
# 交叉检验
def cross_val(model_object, data, label):
    '''
    通过交叉检验计算每次增量学习后的模型得分
    :param model_object: 每次增量学习后的模型对象
    :param data: 训练数据集
    :param label: 训练数据集对应的标签
    :return: 交叉检验得分
    '''
    predict_label = model_object.predict(data)  # 预测测试集标签
    score_tmp = round(float(accuracy_score(label, predict_label)),4)  # 计算预测准确率
    return score_tmp

In [15]:
# word to vector
def word_to_vector(data):
    '''
    将训练集文本数据转换为稀疏矩阵
    :param data: 输入的文本列表
    :return: 稀疏矩阵
    '''
    model_vector = HashingVectorizer(alternate_sign=False)  # 建立HashingVectorizer对象
    vector_data = model_vector.fit_transform(data)  # 将输入文本转化为稀疏矩阵
    return vector_data

In [16]:
# label to vecotr
def label_to_vector(label, unique_list):
    '''
    将文本标签转换为向量标签
    :param label: 文本列表
    :unique_list: 唯一值列表
    :return: 向量标签列表
    '''
    for each_index, each_data in enumerate(label):  # 循环读取每个标签的索引及对应值
        label[each_index] = unique_list.index(each_data)  # 将值替换为其索引
    return label

### 读取数据

In [17]:
# 解压缩文件
if not os.path.exists('./news_data'):  # 如果不存在数据目录，则先解压数据文件
    print('extract data from news_data.tar.gz...')
    with tarfile.open('news_data.tar.gz') as tar:  # 打开tar.gz压缩包对象
        names = tar.getnames()  # 获得压缩包内的每个文件对象的名称
        for name in names:  # 循环读出每个文件
            tar.extract(name, path='./')  # 将文件解压到指定目录

In [18]:
# 定义对象
all_content = []  # 列表，用于存储所有训练集的文本内容
all_label = []  # 列表，用于存储所有训练集的标签
score_list = []  # 列表，用于存储每次交叉检验得分
pre_list = []  # 列表，用于存储每次增量计算后的预测标签
unique_list = ['sports', 'house', 'news']  # 标签唯一值列表
model_nb = MultinomialNB()  # 建立MultinomialNB模型对象

### 交叉检验和预测数据集预处理

In [19]:
# 交叉检验集
with open('test_sets.txt', encoding='utf-8') as f:
    test_data = f.read()
test_content, test_label = data_parse(test_data)  # 解析文本内容和标签
test_data_vector = word_to_vector(test_content)  # 将文本内容向量化
test_label_vecotr = label_to_vector(test_label, unique_list)  # 将标签内容向量化

# 预测集
with open('article.txt', encoding='utf-8') as f:
    new_data = f.read()
new_content, new_label = data_parse(new_data)  # 解析文本内容和标签
new_data_vector = word_to_vector(new_content)  # 将文本内容向量化

### 增量学习

In [20]:
print('{:*^60}'.format('incremental learning...'))
for root, dirs, files in os.walk('./news_data'):  # 分别读取遍历目录下的根目录、子目录和文件列表
    for file in files:  # 读取每个文件
        file_name = os.path.join(root, file)  # 将目录路径与文件名合并为带有完整路径的文件名
        print('training file: %s' % file)
        # 增量训练
        with open(file_name, encoding='utf-8') as f:  # 以只读方式打开文件
            data = f.read()  # 读取文件内容
        content, label = data_parse(data)  # 解析文本内容和标签
        data_vector = word_to_vector(content)  # 将文本内容向量化
        label_vecotr = label_to_vector(label, unique_list)  # 将标签内容向量化
        model_nb.partial_fit(np.abs(data_vector), label_vecotr, classes=np.array([0, 1, 2]))  # 增量学习
        # 交叉检验
        score_list.append(cross_val(model_nb, test_data_vector, test_label_vecotr))  # 将交叉检验结果存入列表
        # 增量预测
        predict_y = model_nb.predict(new_data_vector)  # 预测内容标签
        pre_list.append(predict_y.tolist())

******************incremental learning...*******************
training file: news.sohunews.010806.txt
training file: news.sohunews.020806.txt
training file: news.sohunews.030806.txt
training file: news.sohunews.040806.txt
training file: news.sohunews.050806.txt
training file: news.sohunews.060806.txt
training file: news.sohunews.070806.txt
training file: news.sohunews.080806.txt
training file: news.sohunews.110806.txt
training file: news.sohunews.120806.txt


In [21]:
print('{:*^60}'.format('cross validation score:'))
print(score_list)  # 打印输出每次交叉检验得分
print('{:*^60}'.format('predicted labels:'))
print(pre_list)  # 打印输出每次预测标签索引值
print('{:*^60}'.format('true labels:'))
print(new_label)  # 打印输出正确的标签值

******************cross validation score:*******************
[0.8707, 0.9013, 0.9067, 0.9088, 0.9104, 0.912, 0.9147, 0.9147, 0.9147, 0.9158]
*********************predicted labels:**********************
[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
************************true labels:************************
['sports']
